In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import tabula
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import wikipediaapi

### Replace punctuation by space so it can be accessed as the suffix for wikipedia 

In [3]:
df_updated_bik_rg = pd.read_csv('updated_bik_rg.csv')
df_updated_bik_rg['university/institution'].fillna(value='missing', inplace=True)

temporary_list = list(df_updated_bik_rg.loc[:,'university/institution'])
new_list = []
for ele in temporary_list:
    if ele != 'missing':
        new_list.append(ele.replace("-", " ").replace(",", " ").replace("-", " ").replace("(", " ").replace(")", " "))
    else:
        new_list.append(ele)

In [4]:
df_updated_bik_rg['university/institution'] = new_list
df_updated_bik_rg['university/institution'] = new_list
df_updated_one = df_updated_bik_rg

### Build url searching list

In [5]:
base_url = 'https://en.wikipedia.org/wiki/'
urls = []

for name in new_list:
    if name != "missing":
        suffix = name.strip().replace(" ", "_")
        url = base_url + suffix
        urls.append(url)
    else:
        urls.append("missing_url")

In [6]:
def get_college_location(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        location_elem = soup.find('div', class_='country-name')
        if location_elem:
            return location_elem.get_text(strip=True)
        else:
            return "Location information not found"
    else:
        return "Failed to fetch page"

### Web Scrapping

In [7]:
college_locations = []
for url in urls:
    if url != "missing_url":
        location = get_college_location(url)
        if location == "Location information not found":
            college_locations.append("missing_location")
        else:
            college_locations.append(location)
    else:
        college_locations.append("missing_location")

df_updated_one['Location_List'] = college_locations

In [8]:
df_updated_one['Location_List']

0                   Germany
1             United States
2             United States
3                      U.S.
4             United States
               ...         
209    Failed to fetch page
210    Failed to fetch page
211           United States
212        missing_location
213                   China
Name: Location_List, Length: 214, dtype: object

### Clean some country name by programming

In [9]:
def get_cleaned_name(name):
    if name in ['U.S.','United States[11][note 3]','US','United States]']:
        return "United States"
    elif name in ['Scotland,UK','Northern Ireland','Wales']:
        return "United Kingdom"
    elif name == 'AustraliaUnited Arab EmiratesSingaporeMyanmar':
        return 'Australia'
    elif name == 'Canada[a]':
        return 'Canada'
    elif name in [ "People's Republic of China",'CHN']:
        return "China"
    else:
        return name

In [10]:
df_updated_one['Location_List'] = df_updated_one['Location_List'].apply(get_cleaned_name)

In [11]:
df_updated_one['Location_List']

0                   Germany
1             United States
2             United States
3             United States
4             United States
               ...         
209    Failed to fetch page
210    Failed to fetch page
211           United States
212        missing_location
213                   China
Name: Location_List, Length: 214, dtype: object